In [ ]:
!pip install qutip

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

# Set Up + Useful Functions

In [9]:
zero_ket = basis(2,0)
zero_rho = zero_ket * zero_ket.dag()
I = lambda: qeye(2)

def initial_rho(num_qubits):
  return tensor([zero_rho for i in range (num_qubits)])

def bell_state(state):
  """
    returns requested bell state
  """
  if state == 'phi+':
      phi_plus_ket = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_plus_ket * phi_plus_ket.dag()

  elif state == 'phi-':
      phi_minus_ket = (tensor(basis(2, 0), basis(2, 0)) - tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_minus_ket * phi_minus_ket.dag()

  elif state == 'psi+':
      psi_plus_ket = (tensor(basis(2, 0), basis(2, 1)) + tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_plus_ket * psi_plus_ket.dag()

  elif state == 'psi-':
      psi_minus_ket = (tensor(basis(2, 0), basis(2, 1)) - tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_minus_ket * psi_minus_ket.dag()

  else:
      raise ValueError("Invalid Bell state label")


def pad_op_to_left(op, num_identities):
  if num_identities > 0:
    left_op = I()
    for i in range(num_identities - 1):
      left_op = tensor(left_op, I())
    return tensor(left_op, op)
  else:
    return op

def pad_op_to_right(op, num_identities):
  if num_identities > 0:
    right_op = I()
    for i in range(num_identities - 1):
      right_op = tensor(right_op, I())
    return tensor(op, right_op)
  else:
    return op


In [10]:
def entanglement_generation_with_ptracing_and_dephasing(rho_initial, q1, q2, F_initial):
  """
    This function generates entanglement across 2 neighbouring repeater stations
    It does this by first creating a bell pair across 2 qubits, padding this out
    with identities so that it matches the size of the network, then it multiplies
    this with the initial rho of the network.

    This therefore assumes that the initial state of the qubits it generates
    entanglement accross is 0. It will be incorrect otherwise.
  """
  # SET UP
  num_qubits_left = q1
  num_qubits_right = (num_qubits - 1) - q2
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]


  # GENERATING ENTANGLEMENT
  # initial density matrix of bell pair, taking into account errors initial fidelity
  rho = F_initial * bell_state('phi+') + ((1 - F_initial) / 3) * (bell_state('phi-') +
                                                                       bell_state('psi+') +
                                                                       bell_state('psi-'))
  if len(qubits_left_list) != 0:
    rho_qubits_left = rho_initial.ptrace(qubits_left_list)
    rho = tensor(rho_qubits_left, rho)

  if len(qubits_right_list) != 0:
    rho_qubits_right = rho_initial.ptrace(qubits_right_list)
    rho = tensor(rho, rho_qubits_right)


  # ADDING MEMORY NOISE
  T_dp = 0.001 # dephasing time

  # dephasing channel
  def dephasing_channel(rho, t, left_padding, right_padding):
    dp_prob = (1 - np.exp(-t / T_dp)) / 2
    dp_op = np.sqrt(1 - dp_prob) * qeye(2) + np.sqrt(dp_prob) * sigmaz()
    dp_op = pad_op_to_left(dp_op, left_padding)
    dp_op = pad_op_to_right(dp_op, right_padding)
    rho_t = dp_op * rho * dp_op.dag()
    return rho_t

  # dephasing channel for q1
  rho = dephasing_channel(rho, 2 * d/c, num_qubits_left, num_qubits_right + 1)

  # dephasing channel for q2
  rho = dephasing_channel(rho, d/c, num_qubits_left + 1, num_qubits_right)

  return rho

# Running the Simulation

In [11]:
# Printing partial traces
num_qubits = 4
total_L = 100
d = total_L/(num_qubits - 1)
c = 2e8 # speed of light in fiber optic
rho = initial_rho(num_qubits)

rho = entanglement_generation_with_ptracing_and_dephasing(rho_initial = rho, q1 = 0, q2 = 1, F_initial = 1)

for i in range(num_qubits):
  print(f'Qubit{i}\n', rho.ptrace(i).full(), '\n')

Qubit0
 [[0.52227139+0.j 0.        +0.j]
 [0.        +0.j 0.4781999 +0.j]] 

Qubit1
 [[0.52227139+0.j 0.        +0.j]
 [0.        +0.j 0.4781999 +0.j]] 

Qubit2
 [[1.00047129+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 

Qubit3
 [[1.00047129+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 

